In [1]:
from bs4 import BeautifulSoup
import requests
import re
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

import torch as t
from torch.utils import data
from torchvision import transforms as tsf
import scipy.misc
import torch
from pathlib import Path



from PIL import Image
import skimage
from skimage import io

In [5]:
meme_links = list()
meme_links.append('/meme/First-World-Problems?sort=top-2019')
meme_links.append('/meme/Grandma-Finds-The-Internet?sort=top-2019')
meme_links.append('/meme/Success-Kid?sort=top-2019')
meme_links.append('/meme/Leonardo-Dicaprio-Cheers?sort=top-2019')
meme_links.append('/meme/That-Would-Be-Great?sort=top-2019')
meme_links.append('/meme/Creepy-Condescending-Wonka?sort=top-2019')
meme_links.append('/meme/Disaster-Girl?sort=top-2019')
meme_links.append('/meme/Futurama-Fry?sort=top-2019')
meme_links.append('/meme/One-Does-Not-Simply?sort=top-2019')

In [6]:
image_names = ['boromir_memes/'+'First-World-Problems.jpg','boromir_memes/Grandma-Finds-The-Internet.jpg','boromir_memes/Success-Kid.jpg',
              'boromir_memes/'+'Leonardo-Dicaprio-Cheers.jpg','boromir_memes/That-Would-Be-Great.jpg','boromir_memes/'+'Creepy-Condescending-Wonka.jpg',
              'boromir_memes/'+'Disaster-Girl.jpg','boromir_memes/'+'Futurama-Fry.jpg','boromir_memes/'+'One-Does-Not-Simply.jpg']

In [7]:
images = []
for name in image_names:
    input_image = Image.open(name)
    preprocess = tsf.Compose([
        tsf.ToTensor(),
        tsf.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    input_tensor = preprocess(input_image)
    input_batch = input_tensor.unsqueeze(0) 
    
    images.append(input_batch)

In [8]:
#mapping meme name to image
desc =list()
desc.append(('First World Problems',images[0]))
desc.append(('Grandma Finds The Internet',images[1]))
desc.append(('Success Kid',images[2]))
desc.append(('This is to all',images[3]))
desc.append(('That would be great',images[4]))
desc.append(('Creepy Condescending Wonka',images[5]))
desc.append(('Disaster Girl',images[6]))
desc.append(("NOT SURE IF",images[7]))
desc.append(("ONE DOES NOT SIMPLY",images[8]))


In [9]:
#parsing

datasets = list()
i = 0
for link in meme_links:
    dataset = list()

    while link is not None:
        html_1 = requests.get('https://imgflip.com'+link)
        if i % 10 == 0:
            print('{} links proccessed'.format(i))
        i += 1
        if html_1.status_code != 200:
            print('ERROR')
            break
        soup = BeautifulSoup(html_1.text, 'html.parser')
        lef = soup.find("div", {"id": "base-left"})
        img = lef.find_all('img')
        units = lef.find_all(class_ ='base-unit clearfix')
        for u in units:
            if u.find('img')  and u.find('img')['alt'] != '':
                alt_tag = u.find('img')['alt'].split('|')[1].strip()
                dataset.append(alt_tag)
        link = soup.find(class_ = 'pager-next l but')
        if link:
            link = link['href']

    datasets.append(dataset)

0 links proccessed
10 links proccessed
20 links proccessed
30 links proccessed
40 links proccessed
50 links proccessed
60 links proccessed
70 links proccessed
80 links proccessed
90 links proccessed
100 links proccessed
110 links proccessed
120 links proccessed
130 links proccessed
140 links proccessed
150 links proccessed
160 links proccessed
170 links proccessed
180 links proccessed
190 links proccessed
200 links proccessed
210 links proccessed
220 links proccessed
230 links proccessed
240 links proccessed
250 links proccessed
260 links proccessed
270 links proccessed
280 links proccessed
290 links proccessed
300 links proccessed
310 links proccessed
320 links proccessed
330 links proccessed
340 links proccessed
350 links proccessed
360 links proccessed
370 links proccessed
380 links proccessed
390 links proccessed
400 links proccessed
410 links proccessed
420 links proccessed
430 links proccessed
440 links proccessed
450 links proccessed
460 links proccessed
470 links proccessed
480

In [11]:
pic_id = 0
lines = list()
for pic_id, dataset in enumerate(datasets):
    for text in dataset:
        lines.append((pic_id, text))

In [12]:
import numpy as np
import re
from torch import nn
from torch.autograd import Variable
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
import sys
from IPython.display import clear_output
import tqdm
import progressbar

import torchvision
import torch
import pickle
%matplotlib inline

In [13]:
all_words = set()
with open('glove.6B.300d.txt', 'rb') as f:
    for l in f:
        line = l.decode().split()
        if line[0] not in all_words:
            all_words.add(line[0])
        

In [14]:
new_lines = list()
for line in lines:
    line1 = re.sub(r'[^; \w]+', '', line[1])
    k = line1.find(';')
    if k > 0:
        line1 = line1[:k] +' ' + line1[k:]
    fl = 0
    for k in line1.split(' '):
        if k.lower() not in all_words:
            fl = 1
    if not fl:
        new_lines.append((line[0],line1))   

In [15]:
lines = new_lines

In [16]:
class Vocabulary:
    def __init__(self, sentences):
        self.all_characters = set()
        for line in sentences:
            line = re.sub(r'[^; \w]+', '', line[1])
            for k in line.split(' '):
                if k.lower() not in self.all_characters:
                    self.all_characters.add(k.lower())
        self.all_characters = list(sorted(self.all_characters))+['<eos>', '<go>']
        self.char_to_id = {x[1]:x[0] for x in enumerate(self.all_characters)}
        self.id_to_char = {x[0]:x[1] for x in enumerate(self.all_characters)}
        self.size = len(self.all_characters)

    def encode(self, line):
        return [self.char_to_id[x] for x in line]
    
    def decode(self, tokens):
        return ''.join([self.id_to_char[x] for x in tokens])

In [17]:
import pickle
with open('linesREALTOPnew3.pickle', 'rb') as f:
    lines =  pickle.load(f) 

In [18]:
vocab = Vocabulary(lines)

In [11]:
words_embs = dict()

with open('glove.6B.300d.txt', 'rb') as f:
    for l in f:
        line = l.decode().split()
        word = line[0]
        if word in vocab.all_characters:
            words_embs[vocab.char_to_id[word]] = torch.FloatTensor(np.array(line[1:]).astype(np.float))

In [ ]:
model = torch.hub.load('pytorch/vision:v0.6.0', 'inception_v3', pretrained=True)
model.eval()
model.fc = torch.nn.Sequential()

In [12]:
#encoder
class EncoderRNN1(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers=1):
        super(EncoderRNN1, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(input_size, hidden_size * 2, n_layers, batch_first=False)
        self.lin = nn.Linear(2048 + 300, hidden_size)
        self.cnn = model
    def forward(self, input_pair, hidden):
        img = input_pair[1]
        word_inputs = input_pair[0]
        embedded = words_embs[word_inputs[0].item()]
        seq_len = len(word_inputs)
        for i in range(1, len(word_inputs)):
            embedded = words_embs[word_inputs[i].item()] + embedded
        embedded = embedded / seq_len
        with torch.no_grad():
            img = model(img)
        embedded = torch.cat((img.squeeze(), embedded), 0)
        embedded = embedded.view(1, 1, -1)
        hidden = self.lin(embedded)
        return  hidden

    def init_hidden(self):
        hidden = Variable(torch.zeros(self.n_layers, 1, self.hidden_size))
        return hidden

In [13]:
import torch.nn.functional as F

In [14]:
class Attn():
    pass

In [15]:
#decoder
class AttnDecoderRNN1(nn.Module):
    def __init__(self, hidden_size, output_size, n_layers=1, dropout_p=0.1):
        super(AttnDecoderRNN1, self).__init__()
        
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout_p = dropout_p
        
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size , n_layers, batch_first=False)
        self.out = nn.Linear(hidden_size , output_size)
    
    
    def forward(self, word_input,  last_hidden, l_c):
        
        global words_embs
        if (len(words_embs.get(word_input.item(), torch.LongTensor([0]))) != 1):
            word_embedded =words_embs[word_input.item()].view(1, 1, -1) # S=1 x B x N
        else:
            word_embedded = self.embedding(word_input).view(1, 1, -1)
        
        rnn_output, (hidden,c) = self.lstm(word_embedded, (last_hidden, l_c))

        m = nn.Dropout(p=self.dropout_p)
        rnn_output = rnn_output.squeeze(0) # S=1 x B x N -> B x N
        rnn_output = m(rnn_output)
        output = F.log_softmax(self.out(rnn_output))
        
        return output, (hidden, c)

In [ ]:
def create_pair(line,vocab):
    ans = list()
    ans1 = list()
    line11 = line[1]
    ans.append(vocab.char_to_id['<go>'])
    for k in line11.split(' '):
        ans.append(vocab.char_to_id[k.lower()])
    line1 = desc[line[0]][0]
    img = desc[line[0]][1]
    for k in line1.split(' '):
        ans1.append(vocab.char_to_id[k.lower()])
    ans.append(vocab.char_to_id['<eos>'])
    ans =torch.LongTensor(ans)
    ans1 = torch.LongTensor(ans1)

    return ((ans1,img), ans)

In [16]:
with open('model.pickle', 'rb') as f:
    model =  pickle.load(f) 

In [17]:
hidden_size = 300
n_layers = 1
dropout_p = 0

encoder = EncoderRNN1(vocab.size, hidden_size, n_layers)
decoder = AttnDecoderRNN1(hidden_size, vocab.size, n_layers, dropout_p=dropout_p)


learning_rate = 0.0005
encoder_optimizer = torch.optim.Adam(encoder.parameters(), lr=learning_rate, weight_decay = 1e-5)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=learning_rate,weight_decay = 1e-5)
criterion = nn.NLLLoss()

In [ ]:
teacher_forcing_ratio = 0.75
clip = 5.0

In [ ]:
#training
def train1(input_variable, target_variable, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length, vocab):

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    loss = 0 

    input_length = input_variable[0].size()[0]
    target_length = target_variable.size()[0]

    encoder_hidden = encoder.init_hidden()
    encoder_hidden = encoder(input_variable, encoder_hidden)
    
    decoder_input = torch.Tensor([vocab.char_to_id['<go>']]).long()
    decoder_context = Variable(torch.zeros(1, decoder.hidden_size))
    decoder_hidden = encoder_hidden
    dec_c = Variable(torch.zeros(1, decoder.hidden_size )).view(1, 1, -1)
    use_teacher_forcing = random.random() < teacher_forcing_ratio
    if use_teacher_forcing:
        for di in range(target_length):
            decoder_output, (decoder_hidden,dec_c) = decoder(decoder_input,  decoder_hidden,dec_c)
            loss += criterion(decoder_output, target_variable[di].unsqueeze(0))
            decoder_input = target_variable[di]
    else:
        for di in range(target_length):
            decoder_output, (decoder_hidden,dec_c) = decoder(decoder_input,  decoder_hidden,dec_c)
            loss += criterion(decoder_output, target_variable[di].unsqueeze(0))
            topv, topi = decoder_output.data.topk(1)
            ni = topi[0][0]
            decoder_input = Variable(torch.LongTensor([[ni]]))
            if ni == vocab.char_to_id['<eos>']: break
            decoder_input = Variable(torch.LongTensor([[ni]])) # Chosen word is next input)
    loss.backward()
    torch.nn.utils.clip_grad_norm(encoder.parameters(), clip)
    torch.nn.utils.clip_grad_norm(decoder.parameters(), clip)
    encoder_optimizer.step()
    decoder_optimizer.step()
    
    return loss.item()  / target_length

In [ ]:
from copy import deepcopy
import random

import time
n_epochs = 30000000
plot_every = 50
print_every = 50

start = time.time()
plot_losses = []
print_loss_total = 0 
plot_loss_total = 0 

In [ ]:
#training
from tqdm import tqdm
for epoch in range(1, n_epochs + 1):
    
    training_pair = create_pair(random.choice(lines), vocab)
    input_variable = training_pair[0]
    target_variable = training_pair[1]

    loss = train1(input_variable, target_variable, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion,1000, vocab)
    print_loss_total += loss
    plot_loss_total += loss

    if epoch == 0: continue
    if epoch % 55555 == 0:
        if teacher_forcing_ratio > 0.5:
            teacher_forcing_ratio -= 0.1
    if epoch % print_every == 0:
        print_loss_avg = print_loss_total / print_every
        print_loss_total = 0
        print_summary = ' (%d %d%%) %.4f' % ( epoch, epoch / n_epochs * 100, print_loss_avg)
        print(print_summary)

    if epoch % plot_every == 0:
        plot_loss_avg = plot_loss_total / plot_every
        plot_losses.append(plot_loss_avg)
        plot_loss_total = 0

In [18]:
import pickle 
with open('enctopREALTOPnew3.pickle', 'rb') as f:
    encoder = pickle.load(f)
with open('dectopREALTOPnew3.pickle', 'rb') as f:
    decoder= pickle.load(f)
with open('vocabREALTOPnew3.pickle', 'rb') as f:
    vocab = pickle.load(f)
with open('decoptREALTOPnew3.pickle', 'rb') as f:
    decoder_optimizer = pickle.load(f)
with open('encopttREALTOPnew3.pickle', 'rb') as f:
    encoder_optimizer = pickle.load( f)
with open('datasetsREALTOPnew3.pickle', 'rb') as f:
    datasets =  pickle.load(f)
with open('datasetsREALTOPnew3.pickle', 'rb') as f:
    datasets =  pickle.load(f)
with open('linesREALTOPnew3.pickle', 'rb') as f:
    lines =  pickle.load(f) 
with open('model.pickle', 'rb') as f:
    model =  pickle.load(f) 

In [19]:
def gen(pic_id, encoder, decoder, encoder_optimizer, decoder_optimizer, max_length, vocab):
    ans = list()

    line1 = desc[pic_id][0]
    for k in line1.split(' '):
        ans.append(vocab.char_to_id[k.lower()])
    ans =torch.LongTensor(ans)
    img = desc[pic_id][1]

    encoder_hidden = encoder.init_hidden()
    encoder_hidden = encoder((ans,img), encoder_hidden)
    
    decoder_input = torch.Tensor([vocab.char_to_id['<go>']]).long()
    dec_c = Variable(torch.zeros(1, decoder.hidden_size )).view(1, 1, -1)
    decoder_hidden =  encoder_hidden                                                          
    result = str()
    for di in range(1000):
        decoder_output, (decoder_hidden,dec_c) = decoder(decoder_input,  decoder_hidden,dec_c)
        hh = decoder_output[0].clone().detach().numpy()
        sampler = torch.distributions.Categorical(torch.FloatTensor(np.exp(hh)))
        ni = sampler.sample().data.numpy().item()
        decoder_input = Variable(torch.LongTensor([[ni]]))
        if ni == vocab.char_to_id['<eos>']: break
        decoder_input = Variable(torch.LongTensor([[ni]])) 
        result += (vocab.id_to_char[ni]+' ')
    print(result[5:].upper())

In [22]:
pic_id = 4
gen(pic_id, encoder, decoder, encoder_optimizer, decoder_optimizer, 1000, vocab)

IF LIVE COULD STOP MAKING WIMP ; SKIP STUPID 


/home/rodion/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
